<p>&nbsp; <br>I previously explained that<b> in the world of monitoring, </b><a href="/docs/05_lessdata"><b>Less Data is More Data</b></a>.</p>
  
<p>The goal, again, is to avoid being distracted by a bunch of information that doesn’t matter. Your job is to find and fix the real issues, as quickly as possible.</p>
  
<p>It’s only natural that during the finding phase you might wish to focus in on some particular class of user, or application, or whatever. And that means <b>filters</b>.</p>
  
<p>By default, Who is Active filters out any session that’s not active or that doesn’t have an active request. It also filters out information about the session running Who is Active—i.e., you. The idea is that you’re probably not interested in what Who is Active is up to unless you either have way too much time on your hands, your name is Adam Machanic, or both.</p>
  
<p>If you would like to change any of the default filtered behavior, you can do so using a few options that I introduced in the Less Data is More Data post:</p>
  
<ul>   
<li><b>@show_sleeping_spids</b> defaults to a value of <b>1</b>, which means that the only sleeping sessions that will be shown are those that have an open transaction. If you would like to see <i>all</i> sleeping sessions, set the parameter to a value of <b>2</b>. And if you wouldn’t like to see any, regardless of whether there is an open transaction, set it to <b>0</b>. </li>
    
<li><b>@show_system_spids</b> can be set to <b>1</b> to capture information about background and system processes. It’s generally not very interesting to look at these, with the exception of seeing activity being done by Service Broker activation procedures—and those sessions are already shown by default in newer versions of by Who is Active. </li>
    
<li><b>@show_own_spid</b> can be set to <b>1</b> to capture information about the session you’re running Who is Active from. You’ll see a small chunk of the code that’s running, and it won’t be very insightful, but I’ve included the option for completeness. </li>
 </ul>
  
<p>Beyond the default filters, Who is Active supports the ability to do text filtering on various columns. To see how Who is Active deals with these text filters, let’s take a look at a sample set that might be returned by Who is Active:</p>
  
![F8_lots_of_activity](img/F8_lots_of_activity.jpg)

<p>This <i>blat</i> guy sure is running a lot of queries! I’m much more interested in what Adam is up to (go figure). To reduce the set to see only Adam’s activity, I can employ a filter:</p>
  
<blockquote>   
<p><font face="Courier New">EXEC sp_WhoIsActive        <br>&nbsp;&nbsp;&nbsp; @filter_type = 'login',         <br>&nbsp;&nbsp;&nbsp; @filter = 'Adam03\Adam'</font></p>
 </blockquote>

![F8_just_adam](img/F8_just_adam.jpg)

<p>This filter gives me an exact match on “Adam03\Adam” in the [login_name] column. And that’s great, but a lot more can be done.</p>
  
<p><b>Who is Active supports five filter types</b>:</p>
  
<ul>   
<li>“session” filters on the [session_id] column </li>
    
<li>“program” filters on the [program_name] column </li>
    
<li>“database” filters on the [database_name] column </li>
    
<li>“login” filters on the [login_name] column </li>
    
<li>“host” filters on the [host_name] column </li>
 </ul>
  
<p>Session is special because of the five filter types it is the only one that is numeric. “Filtering” using the “session” filter on a value of “0” returns all of the sessions to be shown based on whatever other filter options you’re working with. A non-zero value will return information only about the session you’ve specified—again, subject to other filter options. This means that if you’re looking for information on session_id 96, but that session is sleeping, and you have @show_sleeping_spids set to 0, you’re not going to see any information. <b>Always remember that the filters are additive</b>. </p>
  
<p>The other four filter types are quite a bit more flexible, because they are pure text filters. And <b>text filters support wildcards</b>. The same kinds of wildcards that you can use for LIKE or PATINDEX. Meaning that you can write the above query in a number of ways, depending on how flexible you would like to be with which sessions are returned:</p>
  
<blockquote>   
<p><font face="Courier New">EXEC sp_WhoIsActive       <br>&nbsp;&nbsp;&nbsp; @filter_type = 'login',        <br>&nbsp;&nbsp;&nbsp; @filter = 'A%';</font></p>
    
<p><font face="Courier New">EXEC sp_WhoIsActive       <br>&nbsp;&nbsp;&nbsp; @filter_type = 'login',        <br>&nbsp;&nbsp;&nbsp; @filter = '%dam';</font></p>
    
<p><font face="Courier New">EXEC sp_WhoIsActive       <br>&nbsp;&nbsp;&nbsp; @filter_type = 'login',        <br>&nbsp;&nbsp;&nbsp; @filter = 'A[de]am03\[^B]dam';</font>      <br></p>
 </blockquote>
  
<p><b>Another option is to ask Who is Active to <i>not </i>show certain sessions</b>. This is referred to as a “not-filter,” and works the same way. If you want to see everything <i>except</i> what <i>blat</i> is up to, you can do:</p>
  
<blockquote>   
<p><font face="Courier New">EXEC sp_WhoIsActive       <br>&nbsp;&nbsp;&nbsp; @not_filter_type = 'login',        <br>&nbsp;&nbsp;&nbsp; @not_filter = 'blat';</font></p>
 </blockquote>
  
<p>The not-filter option was added after some great feedback from Michelle Ufford, who was trying to use Who is Active on a system that was always busy running a huge number of automated jobs and processes. Since all of these had been running for quite some time and were rather uninteresting, Michelle wanted to see only the other, non-automated activity on the system. Not-filters support the same filter types as normal text filters, and also support wildcards. And of course, both filter types can be combined:</p>
  
<blockquote>   
<p><font face="Courier New">EXEC sp_WhoIsActive       <br>&nbsp;&nbsp;&nbsp; @filter_type = 'login',        <br>&nbsp;&nbsp;&nbsp; @filter = 'A%',        <br>&nbsp;&nbsp;&nbsp; @not_filter_type = 'login',        <br>&nbsp;&nbsp;&nbsp; @not_filter = 'b%';</font></p>
 </blockquote>
  
<p>The current set of Who is Active filters are, in my opinion, quite flexible, but I don’t feel completely satisfied. My goal is to eventually convert the entire filter mechanism to an expression language that supports arbitrarily complex filters. This would, of course, also <a href="/docs/02_design">have to be 100% injection safe, and perform extremely well</a>, so it’s a bit of a dream at the moment. But it’s an interesting problem, so I’ll continue to work on it and perhaps I’ll crack it in a future version of Who is Active.</p>